In [1]:
%pylab inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import data
import submit

In [3]:
koubei = data.koubei()

In [4]:
taobao = data.taobao()

In [5]:
tk = taobao[taobao.user_id.isin(koubei.user_id)]

In [6]:
tk.head()

,user_id,seller_id,item_id,category_id,action_id,timestamp
7,1652831,589,1533782,1,0,20150702
9,43130,4058,1745953,1,0,20151019
23,1651174,4058,1745953,1,0,20151010
33,1622079,4058,1745953,1,0,20151002
46,43130,4058,1745953,1,0,20151019


In [7]:
print('总的类别的数目：%d' % tk.category_id.nunique())
print('口碑与淘宝链接后的类别数目：%d'% taobao.category_id.nunique())

总的类别的数目：72
口碑与淘宝链接后的类别数目：72


In [17]:
hotk = tk.groupby(['user_id', 'category_id']).agg(dict(action_id=lambda d: sum((d+1)**2)))

In [18]:
hotk.head()

action_id
user_id category_id           
4       1                    4
        10                   8
        18                   4
        34                   4
        42                   1

In [19]:
hotkc = hotk.reset_index().pivot('user_id', 'category_id', 'action_id').fillna(0)

In [20]:
hotpc.head()

category_id,1,2,3,4,5,6,7,8,9,10,...,63,64,65,66,67,68,69,70,71,72
user_id,,,,,,,,,,,,,,,,,,,,,
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,7,0,0,0
24,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
32,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


--------

In [8]:
pred = pd.read_csv('data/ijcai2016_koubei_test', header=None, names=['user_id', 'location_id'])

In [9]:
tp = taobao[taobao.user_id.isin(pred.user_id)]

In [10]:
tp.head()

,user_id,seller_id,item_id,category_id,action_id,timestamp
0,1980536,9666,1450952,1,0,20150826
1,1980536,9666,1450952,1,0,20150826
3,1980536,9666,1450952,1,0,20150823
4,183647,9525,578730,1,0,20150711
5,723956,589,28301,1,0,20150825


In [11]:
hotp = tp.groupby(['user_id', 'category_id']).agg(dict(action_id=lambda d: sum((d+1)**2)))

In [12]:
hotp.head()

action_id
user_id category_id           
5       45                   4
17      1                    4
23      1                   13
        34                   5
        38                   5

In [13]:
hotpc = hotp.reset_index().pivot('user_id', 'category_id', 'action_id').fillna(0)

In [14]:
hotpc.head()

category_id,1,2,3,4,5,6,7,8,9,10,...,63,64,65,66,67,68,69,70,71,72
user_id,,,,,,,,,,,,,,,,,,,,,
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,7,0,0,0
24,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
32,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [15]:
hotpc.shape

(366385, 72)

In [16]:
hotpc.to_csv('data/0503_pred_category_sim.csv')

-----------

建立邻域搜索模型

In [21]:
from sklearn import neighbors

In [22]:
knn = neighbors.NearestNeighbors(n_jobs=-1, n_neighbors=10, algorithm='auto')

In [23]:
cols = hotkc.columns

In [24]:
X = hotkc.as_matrix(cols)

In [25]:
P = hotpc.as_matrix(cols)

In [26]:
knn.fit(X)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=-1, n_neighbors=10, p=2, radius=1.0)

In [27]:
from sklearn.externals import joblib

In [29]:
joblib.dump(knn, 'model/knn_cate.sklearn/0503')

['model/knn_cate.sklearn/0503',
 'model/knn_cate.sklearn/0503_01.npy',
 'model/knn_cate.sklearn/0503_02.npy',
 'model/knn_cate.sklearn/0503_03.npy',
 'model/knn_cate.sklearn/0503_04.npy',
 'model/knn_cate.sklearn/0503_05.npy',
 'model/knn_cate.sklearn/0503_06.npy',
 'model/knn_cate.sklearn/0503_07.npy']

In [30]:
nidx = knn.kneighbors(P, return_distance=True)

In [31]:
dist, idx = nidx # only use when return_distance is False.

In [32]:
XU = hotkc.index

In [33]:
PU = hotpc.index

In [34]:
XU[0]

4

In [35]:
PU[0]

5

In [39]:
pd.DataFrame.from_records?

In [40]:
simdf = pd.DataFrame.from_records([
    (PU[i], XU[k], dist[i, j]) for i, r in enumerate(idx) for j, k in enumerate(r)
], columns=['user_id', 'sim_user_id', 'cate_sim_score'])

In [41]:
simdf.set_index('user_id').to_csv('data/u_cate_sim_0504.csv')

In [42]:
simdf.head()

,user_id,sim_user_id,cate_sim_score
0,5,12711,0
1,5,70646,0
2,5,54852,0
3,5,57799,0
4,5,31803,0


In [44]:
len(simdf[simdf.cate_sim_score < 3])

975321

In [45]:
len(simdf)

3663850

---------

连接相似的用户访问过的商店和地点

In [46]:
rk = pd.read_csv('model/koubei_time_agg_0504.csv')

In [47]:
rk.head()

,user_id,location_id,merchant_id,timestamp_agg,timestamp_len,time_agg_rank,len_rank
0,4,254,3527,0.555159,1,1,1
1,22,371,4264,1.372361,2,1,1
2,38,272,820,1.381452,2,1,1
3,42,337,820,1.855724,2,1,1
4,46,348,9236,0.506085,1,1,1


In [48]:
simmer = pd.merge(
    simdf[simdf.cate_sim_score < 3],
    rk.set_index('user_id'),
    left_on='sim_user_id', right_index=True, how='left'
)

In [49]:
simmer.head()

,user_id,sim_user_id,cate_sim_score,location_id,merchant_id,timestamp_agg,timestamp_len,time_agg_rank,len_rank
0,5,12711,0,192,7755,0.562345,1,1,1
1,5,70646,0,34,1456,0.891379,1,1,1
2,5,54852,0,84,1978,0.505168,1,1,1
3,5,57799,0,441,4234,0.605993,1,1,1
4,5,31803,0,348,5231,0.590025,1,1,1


In [50]:
simmer2 = pd.merge(
    simmer,
    simmer.groupby(['user_id', 'merchant_id']).sim_user_id.nunique().reset_index().rename(columns={'sim_user_id': 'sim_users'}),
    on=['user_id', 'merchant_id'], how='left'
)

In [52]:
simmer3 = pd.merge(
    simmer2,
    simmer.groupby(['user_id', 'merchant_id']).location_id.nunique().reset_index().rename(columns={'location_id': 'locations'}),
    on=['user_id', 'merchant_id'], how='left'
)

In [56]:
simmer3.rename(columns={
    'timestamp_agg': 'ut_agg',
    'time_agg_rank': 'ut_agg_rank',
    'timestamp_len': 'ut_len',
    'len_rank': 'ut_len_rank'
}, inplace=True)

In [57]:
simmer3.head()

,user_id,sim_user_id,cate_sim_score,location_id,merchant_id,ut_agg,ut_len,ut_agg_rank,ut_len_rank,sim_users,locations
0,5,12711,0,192,7755,0.562345,1,1,1,2,2
1,5,70646,0,34,1456,0.891379,1,1,1,1,1
2,5,54852,0,84,1978,0.505168,1,1,1,1,1
3,5,57799,0,441,4234,0.605993,1,1,1,1,1
4,5,31803,0,348,5231,0.590025,1,1,1,1,1


In [92]:
simmer2.set_index('user_id').to_csv('data/user_catsim_merchant.csv')

In [59]:
merchant = data.merchant().rename(columns=dict(locations='all_locations'))
merchant['all_locations'] = merchant.all_locations.map(lambda s: len(s.split(':')))

In [60]:
merchant.head()

,merchant_id,budget,all_locations
0,128,331,1
1,3712,100,1
2,1409,100,1
3,3201,100,1
4,8577,100,1


In [61]:
simmer4 = pd.merge(simmer3, merchant, on='merchant_id', how='left')

In [64]:
simmer4.head()

,user_id,sim_user_id,cate_sim_score,location_id,merchant_id,ut_agg,ut_len,ut_agg_rank,ut_len_rank,sim_users,locations,budget,all_locations
0,5,12711,0,192,7755,0.562345,1,1,1,2,2,12223,8
1,5,70646,0,34,1456,0.891379,1,1,1,1,1,100,1
2,5,54852,0,84,1978,0.505168,1,1,1,1,1,2421,3
3,5,57799,0,441,4234,0.605993,1,1,1,1,1,100,1
4,5,31803,0,348,5231,0.590025,1,1,1,1,1,7721,31


In [67]:
simmer5 = pd.merge(
    simmer4, simmer4.groupby(['user_id', 'location_id', 'merchant_id']).ut_agg.sum().reset_index().rename(columns={'ut_agg': 'ut_score'}),
    on=['user_id', 'location_id', 'merchant_id'], how='left'
)

In [68]:
simmer5.head()

,user_id,sim_user_id,cate_sim_score,location_id,merchant_id,ut_agg,ut_len,ut_agg_rank,ut_len_rank,sim_users,locations,budget,all_locations,ut_score
0,5,12711,0,192,7755,0.562345,1,1,1,2,2,12223,8,0.562345
1,5,70646,0,34,1456,0.891379,1,1,1,1,1,100,1,0.891379
2,5,54852,0,84,1978,0.505168,1,1,1,1,1,2421,3,0.505168
3,5,57799,0,441,4234,0.605993,1,1,1,1,1,100,1,0.605993
4,5,31803,0,348,5231,0.590025,1,1,1,1,1,7721,31,0.590025


In [69]:
simmer5['ulm_ut_score_rank'] = simmer5.groupby(['user_id', 'location_id']).ut_score.rank(ascending=False, method='first')

In [72]:
rim = simmer5[simmer5.ulm_ut_score_rank < 5].copy()

In [73]:
rim['m_ut_score_rank'] = rim.groupby('merchant_id').ut_score.rank(ascending=False, method='first')

In [74]:
rim2 = rim[rim.m_ut_score_rank <= rim.budget]

In [89]:
s3 = rim2.groupby(['user_id', 'location_id']).merchant_id.apply(lambda d: ':'.join(str(int(i)) for i in d)).reset_index()

In [80]:
s.head()

user_id  location_id
5        34                  1456
         80                  3715
         188                  334
         192                 7755
         308            7755,8837
Name: merchant_id, dtype: object

In [79]:
import submit

In [81]:
s1 = submit.load('submit/0504_1606.csv')

In [88]:
s2 = s1.groupby(['user_id', 'location_id']).merchant_id.apply(lambda d: ':'.join(str(int(i)) for i in d)).reset_index()

In [95]:
pd.concat([s2, s3[~s3.user_id.isin(s2)]]).set_index(['user_id', 'location_id']).to_csv('submit/0506.csv')

In [92]:
rim3 = rim[rim.m_ut_score_rank <= rim.budget / 4]

In [93]:
s4 = rim3.groupby(['user_id', 'location_id']).merchant_id.apply(lambda d: ':'.join(str(int(i)) for i in d)).reset_index()

In [96]:
pd.concat([s2, s4[~s4.user_id.isin(s2)]]).set_index(['user_id', 'location_id']).merchant_id.to_csv('submit/0507.csv')

---------------

地理位置附近的（关联的）商家，以及根据用户相似度找到的对商家的支持度。

In [98]:
lm = koubei.groupby(['location_id', 'merchant_id']).user_id.nunique().reset_index()
lm.rename(columns={'user_id': 'merchant_users'}, inplace=True)

In [99]:
lm.head()

,location_id,merchant_id,merchant_users
0,1,1225,2
1,1,3222,3
2,1,3780,2
3,1,4417,3
4,1,7547,2


In [101]:
plm = pd.merge(pred, lm, on='location_id', how='left')

In [102]:
plm.head()

,user_id,location_id,merchant_id,merchant_users
0,753792,112,161,35
1,753792,112,571,80
2,753792,112,670,1
3,753792,112,704,12
4,753792,112,820,684


In [105]:
plums = pd.merge(plm, ums, on=['user_id', 'merchant_id'], how='left')

In [106]:
plums.head()

,user_id,location_id,merchant_id,merchant_users,sim_users
0,753792,112,161,35,NaN
1,753792,112,571,80,NaN
2,753792,112,670,1,NaN
3,753792,112,704,12,NaN
4,753792,112,820,684,NaN


In [109]:
count_nonzero(plums[~plums.user_id.isin(koubei.user_id)].sim_users)

18353776

In [112]:
nk = plums[~plums.user_id.isin(koubei.user_id)].dropna()

In [126]:
nk['sim_users_rank'] = nk.groupby(['user_id', 'location_id']).sim_users.rank(ascending=False, method='first')

In [127]:
nk.head()

,user_id,location_id,merchant_id,merchant_users,sim_users,sim_users_rank
387,1279360,254,5231,1406,4,1
439,296576,253,8817,82,1,1
539,362624,265,820,622,1,1
553,362624,265,5231,429,1,2
730,1543040,134,4264,65,1,1


In [128]:
nkm = nk[nk.sim_users_rank < 3]

In [133]:
rec = pd.concat([
    submit.load('submit/0501.csv'),
    nkm[['user_id', 'location_id', 'merchant_id']]
])

In [134]:
rec.head()

,user_id,location_id,merchant_id
0,42,337,820
1,89,188,7667
2,97,84,3538
3,206,83,2466
4,206,83,2606


In [135]:
s = rec.groupby(['user_id', 'location_id']).merchant_id.agg(lambda d: ':'.join(str(int(i)) for i in d))

In [136]:
s.head()

user_id  location_id
5        371                5231
17       11                 8817
32       190            820:2606
42       337                 820
48       461            820:5231
Name: merchant_id, dtype: object

In [137]:
s.to_csv('submit/0502.csv')